# Task
We want to achieve the following workflow:

1. do addition until reaching a certain value

**Note:**

Different from the previous `dflow-recurse.ipynb` example, this example shows how to reuse step from previous workflow. 

First, we will redo what we have done in `dflow-recurse.ipynb`. (`iter` starts from 0, `limit` sets to 3)

Then, we will increase the `limit` to 6. But this time we want to reuse the steps from the first time.

**BONUS**, we can modify the output of a step. So the third task we want to do is. We want to run the workflow from 0 (`iter`) to 6 (`limit`). But this time, we want to achieve it by modifying the output. 

**Why can we do this?**

All the operated workflow are stored in the artifact repository (e.g. minio). Once you define a way to access them, you can find them in the artifact repository and reuse it.

In [1]:
from dflow import InputParameter, Inputs, Step, Steps, Workflow
from dflow.python import OP, OPIO, OPIOSign, PythonOPTemplate

In [2]:
class Plus1(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'iter': int
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'iter': int
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        #we add this block to see effect of reuse
        ###############################
        import time 
        time.sleep(5) 
        ###############################
        
        return OPIO({
            'iter': op_in['iter'] + 1
        })

Now, `Steps` is defined the same as `dflow-recurse.ipynb`.

In [3]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=3)}
    ),
)

This is the step we want to reuse later

In [4]:
addition = Step(name="add",
                template=PythonOPTemplate(Plus1, image="python:3.8"),
                parameters={"iter": steps.inputs.parameters["iter"]},
                key="iter-%s" % steps.inputs.parameters["iter"],  # a unique key
                )

**NOTE**:
In order to reuse it, we need to define a unique key.  

In [5]:
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when="%s < %s" % (addition.outputs.parameters["iter"], steps.inputs.parameters["limit"]),
)

Now, we have all the steps setup. 

## First, original recurse step

In [6]:
steps.add(addition)
steps.add(loop)

wf = Workflow("recurse", steps=steps)
wf.submit();

Workflow has been submitted (ID: recurse-vvxjq, UID: 6dd772ae-11a5-4e93-ada3-424a667bb56a)


## Then, reuse the workflow

Wait the previous workflow to complete

In [7]:
step0 = wf.query_step(key='iter-0')[0]
step1 = wf.query_step(key='iter-1')[0]
step2 = wf.query_step(key='iter-2')[0]

Now change the value in the steps and rerun it. To not make confusion, we just copy the above code to the cell below. 

In [8]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=6)}
    ),
)
addition = Step(name="add",
                template=PythonOPTemplate(Plus1,
                                        image="python:3.8"),
                parameters={"iter": steps.inputs.parameters["iter"]},
                key="iter-%s" % steps.inputs.parameters["iter"]) # do not modify the key!!!!
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when="%s < %s" % (addition.outputs.parameters["iter"], steps.inputs.parameters["limit"]),
)

In [9]:
steps.add(addition)
steps.add(loop)

wf = Workflow("recurse-resubmit", steps=steps)
wf.submit(reuse_step=[step0, step1, step2]);

Workflow has been submitted (ID: recurse-resubmit-qreh8, UID: 9248232a-b9af-4cc1-850a-e203cf3d345f)


## Bonus, we can modify the output of a step.

In [10]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=6)}
    ),
)
addition = Step(name="add",
                template=PythonOPTemplate(Plus1,
                                        image="python:3.8"),
                parameters={"iter": steps.inputs.parameters["iter"]},
                key="iter-%s" % steps.inputs.parameters["iter"]) # do not modify the key!!!!
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when="%s < %s"
    % (addition.outputs.parameters["iter"], steps.inputs.parameters["limit"]),
)

In [11]:
step2.modify_output_parameter("iter", 5)

In [12]:
steps.add(addition)
steps.add(loop)

wf = Workflow("recurse-resubmit", steps=steps)
wf.submit(reuse_step=[step0, step1, step2]);

Workflow has been submitted (ID: recurse-resubmit-nw0xv, UID: 7fae24de-406a-465c-aa30-0066b8119cf1)
